In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

# Genetic Algorithm for Job Shop Scheduling Problem (JSSP)


In [2]:
def parse_dataset(file_path, instance_name):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    start_idx = -1
    for idx, line in enumerate(lines):
        if line.strip() == f"instance {instance_name}": 
            start_idx = idx  #If the line matches "instance {instance_name}", record its position (start_idx).

            break 

    if start_idx == -1: #If the instance isn’t found (start_idx remains -1), raise an error.
        raise ValueError(f"Instance {instance_name} not found in the dataset file.")

    job_data = []
    for idx in range(start_idx + 1, len(lines)):
        line = lines[idx].strip()
        if line.startswith('+') or line == "":
            continue
        try:
            job_count, machine_count = map(int, line.split())
            count_line_idx = idx
            break
        except ValueError:
            continue
    else:
        raise ValueError(f"Failed to find job and machine counts for instance {instance_name}.")

    for line in lines[count_line_idx + 1:]:
        line = line.strip()
        if line.startswith('+') or line == "":
            break
        tasks = list(map(int, line.split()))
        job = [(tasks[i], tasks[i + 1]) for i in range(0, len(tasks), 2)]
        job_data.append(job)  ##Look for the line that specifies the number of jobs  and machines.


    if len(job_data) != job_count:
        raise ValueError(f"Mismatch between declared job count ({job_count}) and parsed jobs ({len(job_data)}).")

    return job_data, machine_count


In [3]:
''''
# Define the data for the Job Shop Scheduling Problem (JSSP)
jobs_data = [
    [(0, 3), (1, 2), (2, 2)],  # Job 0
    [(0, 2), (2, 1), (1, 4)],  # Job 1
    [(1, 4), (2, 3)],          # Job 2
    [(0, 2), (1, 3), (2, 1)],  # Job 3
    [(1, 5), (2, 2), (0, 4)]   # Job 4
]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)
horizon = sum(task[1] for job in jobs_data for task in job) 

'''

"'\n# Define the data for the Job Shop Scheduling Problem (JSSP)\njobs_data = [\n    [(0, 3), (1, 2), (2, 2)],  # Job 0\n    [(0, 2), (2, 1), (1, 4)],  # Job 1\n    [(1, 4), (2, 3)],          # Job 2\n    [(0, 2), (1, 3), (2, 1)],  # Job 3\n    [(1, 5), (2, 2), (0, 4)]   # Job 4\n]\n\nmachines_count = 1 + max(task[0] for job in jobs_data for task in job)\nall_machines = range(machines_count)\nhorizon = sum(task[1] for job in jobs_data for task in job) \n\n"

In [4]:
def repair_chromosome(chromosome, jobs_data):
    # Initialize a task tracker to keep track of the current task index for each job
    task_tracker = {job_id: 0 for job_id in range(len(jobs_data))}
    
    # Initialize machine_times to track the availability of each machine (when it's free to take the next task)
    machine_times = {machine: 0 for job in jobs_data for machine, _ in job}
    repaired_chromosome = []

    # Traverse the chromosome and add tasks while respecting job order
    for job_id, (machine, duration) in chromosome:
        # Check if the current task in the chromosome is valid for this job
        if task_tracker[job_id] < len(jobs_data[job_id]):
            expected_task = jobs_data[job_id][task_tracker[job_id]]

            # Ensure we follow the correct order for this job
            if (machine, duration) == expected_task:
                # Determine the earliest available start time for this machine
                start_time = machine_times[machine]
                
                # Update the machine's availability
                machine_times[machine] = start_time + duration
                
                # Add task to the repaired chromosome
                repaired_chromosome.append((job_id, (machine, duration)))
                task_tracker[job_id] += 1

    # Fill any missing tasks while maintaining order
    for job_id in range(len(jobs_data)):
        # Continue adding tasks for each job until all tasks are completed in the correct order
        while task_tracker[job_id] < len(jobs_data[job_id]):
            task = jobs_data[job_id][task_tracker[job_id]]
            repaired_chromosome.append((job_id, task))
            task_tracker[job_id] += 1

    return repaired_chromosome


In [5]:
import random

def generate_population(jobs_data, population_size):
    # Initialize an empty list to store the generated population
    population = []

    # Generate 'population_size' chromosomes
    for i in range(population_size):
        chromosome = []

        # Create a chromosome by iterating through each job and its tasks
        for job_id, job in enumerate(jobs_data):
            for task in job:
                # Append each task as a tuple of (job_id, (machine, processing_time))
                chromosome.append((job_id, task))

        # Shuffle only the first half of the population to introduce diversity
        if i < population_size // 2:
            #To balance between randomness (exploration) and structure (exploitation).
            random.shuffle(chromosome)

        # Repair the chromosome to ensure it follows JSSP constraints
        chromosome = repair_chromosome(chromosome, jobs_data)

        # Add the repaired chromosome to the population
        population.append(chromosome)

    # Return the final generated population
    return population


In [6]:
# Fitness function: Calculate makespan
def calculate_fitness(chromosome, machines_count, jobs_data):
    machine_times = [0] * machines_count
    job_times = [0] * len(jobs_data)

    for job_id, (machine, duration) in chromosome:
        start_time = max(machine_times[machine], job_times[job_id])
        end_time = start_time + duration
        machine_times[machine] = end_time
        job_times[job_id] = end_time

    return max(machine_times)


In [7]:
# Selection: Tournament selection
def tournament_selection(population, fitness, k=3):
    # Randomly select 'k' individuals from the population along with their fitness scores.
    selected = random.sample(list(zip(population, fitness)), k)
    # Return the individual with the minimum fitness score from the selected sample.
    return min(selected, key=lambda x: x[1])[0]

In [8]:
#     Two-point crossover: Selects two points and swaps the segments between them.
def two_point_crossover(parent1, parent2, crossover_rate, jobs_data):
    if random.random() < crossover_rate:
        point1, point2 = sorted(random.sample(range(len(parent1)), 2))
        offspring1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
        offspring2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
        # Repair internally
        offspring1 = repair_chromosome(offspring1, jobs_data)
        offspring2 = repair_chromosome(offspring2, jobs_data)
    else:
        offspring1, offspring2 = parent1[:], parent2[:]

    return offspring1, offspring2


In [9]:
def uniform_crossover(parent1, parent2, crossover_rate, jobs_data):
    if random.random() < crossover_rate:
        offspring1, offspring2 = [], []
        for gene1, gene2 in zip(parent1, parent2):
            if random.random() < 0.5:
                offspring1.append(gene1)
                offspring2.append(gene2)
            else:
                offspring1.append(gene2)
                offspring2.append(gene1)
        
        # Repair offspring to maintain task order
        offspring1 = repair_chromosome(offspring1, jobs_data)
        offspring2 = repair_chromosome(offspring2, jobs_data)
    else:
        offspring1, offspring2 = parent1[:], parent2[:]

    return offspring1, offspring2


In [10]:
# Mutation: Always One Mutation per Chromosome ,To prevent getting stuck with bad solutions.
def always_one_mutation(chromosome, jobs_data):
    index1 = random.randint(0, len(chromosome) - 1)
    index2 = random.randint(0, len(chromosome) - 1)

    # Ensure that the two tasks belong to different jobs
    if chromosome[index1][0] != chromosome[index2][0]:
        chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]

    # Repair internally to maintain task order
    return repair_chromosome(chromosome, jobs_data)


In [11]:
# Mutation: Each Gene Mutates Independently
def independent_gene_mutation(chromosome, mutation_rate, jobs_data):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            swap_index = random.randint(0, len(chromosome) - 1)
            
            # Ensure that the two tasks belong to different jobs
            if chromosome[i][0] != chromosome[swap_index][0]:
                chromosome[i], chromosome[swap_index] = chromosome[swap_index], chromosome[i]

    # Repair internally to maintain task order
    return repair_chromosome(chromosome, jobs_data)



In [12]:
def genetic_algorithm(jobs_data, population_size=50, generations=100, crossover_rate=0.8, mutation_rate=0.1, elite_count=2):
    machines_count = len(set(machine for job in jobs_data for machine, _ in job))
    population = generate_population(jobs_data, population_size)
    best_solution = None
    best_fitness = float('inf')
    fitness_history = []

    for generation in range(generations):
        # Step 1: Calculate fitness
        fitness = [calculate_fitness(chromosome, machines_count, jobs_data) for chromosome in population]
        sorted_population = [x for _, x in sorted(zip(fitness, population), key=lambda pair: pair[0])]
        sorted_fitness = sorted(fitness)

        # Step 2: Select elites
        elites = sorted_population[:elite_count]
        new_population = elites[:]

        # Step 3: Generate new offspring
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, fitness)
            parent2 = tournament_selection(population, fitness)

            # Step 4: Select crossover technique using if-else
            if random.random() < 0.5:  # 50% chance to pick either crossover method
                child1, child2 = two_point_crossover(parent1, parent2, crossover_rate, jobs_data)
            else:
                child1, child2 = uniform_crossover(parent1, parent2, crossover_rate, jobs_data)

            # Step 5: Mutation using if-else
            for child in [child1, child2]:
                if random.random() < mutation_rate:
                    if random.random() < 0.5:  # 50% chance to pick either mutation method
                        child = independent_gene_mutation(child, mutation_rate, jobs_data)
                    else:
                        child = always_one_mutation(child, jobs_data)


            new_population.append(child1)
            if len(new_population) < population_size:
                new_population.append(child2)

        # Step 8: Replace old population with new one
        population = new_population

        # Step 9: Track best solution
        best_gen_fitness = sorted_fitness[0]
        best_gen_solution = sorted_population[0]

        if best_gen_fitness < best_fitness:
            best_fitness = best_gen_fitness
            best_solution = best_gen_solution

        fitness_history.append(best_fitness)

    # Debugging statements to check types before return
    print(f"Returning types - Best Solution: {type(best_solution)}, Best Fitness: {type(best_fitness)}, Fitness History: {type(fitness_history)}")

    return best_solution, best_fitness, fitness_history


In [13]:
def simulated_annealing(jobs_data, initial_solution, initial_temperature, cooling_rate, max_iterations):
    # Get the total number of machines in the job shop problem
    machines_count = len(set(machine for job in jobs_data for machine, _ in job))

    # Initialize the solution and fitness tracking
    current_solution = initial_solution
    current_fitness = calculate_fitness(current_solution, machines_count, jobs_data)
    best_solution = current_solution
    best_fitness = current_fitness
    temperature = initial_temperature
    fitness_history = [current_fitness]

    for iteration in range(max_iterations):
        # Generate a new neighbor by swapping tasks
        neighbor_solution = get_neighbor_solution(current_solution[:])

        # Repair the neighbor to ensure it respects job order constraints
        repaired_neighbor = repair_solution(neighbor_solution)

        # Calculate the fitness of the repaired neighbor
        new_fitness = calculate_fitness(repaired_neighbor, machines_count, jobs_data)

        # Accept the new solution if it's better, or with a probability based on temperature
        if new_fitness < current_fitness or random.random() < math.exp(-(new_fitness - current_fitness) / temperature):
            current_solution = repaired_neighbor
            current_fitness = new_fitness

            # Update the best solution if the new solution is better
            if new_fitness < best_fitness:
                best_solution = repaired_neighbor
                best_fitness = new_fitness

        # Gradually cool down the temperature
        temperature *= cooling_rate
        fitness_history.append(best_fitness)

        # Stop if the temperature is close to zero
        if temperature < 1e-6:
            break

    return best_solution, best_fitness, fitness_history


In [14]:
def get_neighbor_solution(current_solution):
    # Find possible task swaps between tasks from different jobs
    valid_swaps = [
        (i, j) for i in range(len(current_solution)) 
        for j in range(i + 1, len(current_solution)) 
        if current_solution[i][0] != current_solution[j][0]  # Ensure they belong to different jobs
    ]
    
    # Randomly choose a valid swap and perform it
    if valid_swaps:
        i, j = random.choice(valid_swaps)
        current_solution[i], current_solution[j] = current_solution[j], current_solution[i]
    return current_solution


In [15]:
def repair_solution(neighbor):
    task_tracker = {job_id: 0 for job_id in range(len(jobs_data))}
    machine_availability = [0] * machines_count
    repaired_solution = []

    # Ensure the tasks follow the correct order for each job
    for job_id, (machine, duration) in neighbor:
        if task_tracker[job_id] < len(jobs_data[job_id]) and jobs_data[job_id][task_tracker[job_id]] == (machine, duration):
            start_time = machine_availability[machine]
            repaired_solution.append((job_id, (machine, duration)))
            machine_availability[machine] = start_time + duration
            task_tracker[job_id] += 1

    # Fill in any missing tasks in the correct order
    for job_id in range(len(jobs_data)):
        while task_tracker[job_id] < len(jobs_data[job_id]):
            task = jobs_data[job_id][task_tracker[job_id]]
            repaired_solution.append((job_id, task))
            task_tracker[job_id] += 1

    return repaired_solution


In [16]:
# Main code
file_path = 'jobshop1.txt'
instance_name = 'abz5'

try:
    # Parse dataset
    jobs_data, machines_count = parse_dataset(file_path, instance_name)
    print(f"Parsed Jobs Data: {jobs_data}")
    print(f"Machines Count: {machines_count}")

    # Define GA parameter combinations
    ga_parameter_combinations = [
        {"population_size": 300, "mutation_rate": 0.1, "crossover_rate": 0.8, "elitism": 2},
        {"population_size": 250, "mutation_rate": 0.05, "crossover_rate": 0.7, "elitism": 2},
        {"population_size": 50, "mutation_rate": 0.2, "crossover_rate": 0.5, "elitism": 1},
        {"population_size": 200, "mutation_rate": 0.2, "crossover_rate": 0.9, "elitism": 3},
        {"population_size": 100, "mutation_rate": 0.1, "crossover_rate": 0.6, "elitism": 1},
        {"population_size": 400, "mutation_rate": 0.05, "crossover_rate": 0.9, "elitism": 3},
    ]

    # Store GA results
    ga_results = []

    # Run Genetic Algorithm for each parameter combination
    for params in ga_parameter_combinations:
        print(f"\nRunning GA with parameters: {params}")
        ga_best_solution, ga_best_fitness, ga_fitness_history = genetic_algorithm(
            jobs_data, params["population_size"], 100, params["crossover_rate"],
            params["mutation_rate"], params["elitism"]
        )

        # Save results
        ga_results.append({
            "parameters": params,
            "best_fitness": ga_best_fitness,
            "best_solution": ga_best_solution,
            "fitness_history": ga_fitness_history,
        })

        # Print results for GA
        print(f"GA Results - Parameters: {params}")
        print(f"Best Fitness (Makespan): {ga_best_fitness}")
        print(f"Best Solution: {ga_best_solution}")

        # Plot fitness evolution
        plt.figure()
        plt.plot(ga_fitness_history, label=f"GA - Pop: {params['population_size']}, Mut: {params['mutation_rate']}, X: {params['crossover_rate']}, E: {params['elitism']}")
        plt.xlabel('Generation')
        plt.ylabel('Best Fitness (Makespan)')
        plt.title(f"Fitness Evolution (GA - Pop: {params['population_size']}, Mut: {params['mutation_rate']}, X: {params['crossover_rate']}, E: {params['elitism']})")
        plt.legend()
        plt.savefig(f"fitness_evolution_ga_pop{params['population_size']}_mut{params['mutation_rate']}_xover{params['crossover_rate']}_elitism{params['elitism']}.png")
        plt.close()

    # Define SA parameter combinations
    sa_parameter_combinations = [
        {"initial_temperature": 1000, "cooling_rate": 0.99, "max_iterations": 1000},
        {"initial_temperature": 500, "cooling_rate": 0.95, "max_iterations": 1000},
        {"initial_temperature": 1500, "cooling_rate": 0.98, "max_iterations": 1500},
        {"initial_temperature": 800, "cooling_rate": 0.97, "max_iterations": 1200},
        {"initial_temperature": 1200, "cooling_rate": 0.96, "max_iterations": 1500},
        {"initial_temperature": 700, "cooling_rate": 0.94, "max_iterations": 1100},
    ]

    # Store SA results
    sa_results = []

    # Run Simulated Annealing for each parameter combination
    for params in sa_parameter_combinations:
        print(f"\nRunning SA with parameters: {params}")
    #one solution , first only solution
        sa_initial_solution = generate_population(jobs_data, 1)[0]
        sa_best_solution, sa_best_fitness, sa_fitness_history = simulated_annealing(
            jobs_data, sa_initial_solution, params["initial_temperature"], params["cooling_rate"], params["max_iterations"]
        )

        # Save results
        sa_results.append({
            "parameters": params,
            "best_fitness": sa_best_fitness,
            "best_solution": sa_best_solution,
            "fitness_history": sa_fitness_history,
        })

        # Print results for SA
        print(f"SA Results - Parameters: {params}")
        print(f"Best Fitness (Makespan): {sa_best_fitness}")
        print(f"Best Solution: {sa_best_solution}")

        # Plot fitness evolution
        plt.figure()
        plt.plot(sa_fitness_history, label=f"SA - Temp: {params['initial_temperature']}, Cool: {params['cooling_rate']}, Iter: {params['max_iterations']}")
        plt.xlabel('Iteration')
        plt.ylabel('Best Fitness (Makespan)')
        plt.title(f"Fitness Evolution (SA - Temp: {params['initial_temperature']}, Cool: {params['cooling_rate']}, Iter: {params['max_iterations']})")
        plt.legend()
        plt.savefig(f"fitness_evolution_sa_temp{params['initial_temperature']}_cool{params['cooling_rate']}_iter{params['max_iterations']}.png")
        plt.close()

except Exception as e:
    print(f"Error: {e}")

Parsed Jobs Data: [[(4, 88), (8, 68), (6, 94), (5, 99), (1, 67), (2, 89), (9, 77), (7, 99), (0, 86), (3, 92)], [(5, 72), (3, 50), (6, 69), (4, 75), (2, 94), (8, 66), (0, 92), (1, 82), (7, 94), (9, 63)], [(9, 83), (8, 61), (0, 83), (1, 65), (6, 64), (5, 85), (7, 78), (4, 85), (2, 55), (3, 77)], [(7, 94), (2, 68), (1, 61), (4, 99), (3, 54), (6, 75), (5, 66), (0, 76), (9, 63), (8, 67)], [(3, 69), (4, 88), (9, 82), (8, 95), (0, 99), (2, 67), (6, 95), (5, 68), (7, 67), (1, 86)], [(1, 99), (4, 81), (5, 64), (6, 66), (8, 80), (2, 80), (7, 69), (9, 62), (3, 79), (0, 88)], [(7, 50), (1, 86), (4, 97), (3, 96), (0, 95), (8, 97), (2, 66), (5, 99), (6, 52), (9, 71)], [(4, 98), (6, 73), (3, 82), (2, 51), (1, 71), (5, 94), (7, 85), (0, 62), (8, 95), (9, 79)], [(0, 94), (6, 71), (3, 81), (7, 85), (1, 66), (2, 90), (4, 76), (5, 58), (8, 93), (9, 97)], [(3, 50), (0, 59), (1, 82), (8, 67), (7, 56), (9, 96), (6, 58), (4, 81), (5, 59), (2, 96)]]
Machines Count: 10

Running GA with parameters: {'population_